In [1]:
import pygame
import cv2
import numpy as np
import math

In [2]:
## List of methods to be used
def is_in_rect(x,y,p1,p4):
    f1 = y - p1[1]
    f2 = y - p4[1]
    f3 = x - p1[0]
    f4 = x - p4[0]
    if f1 >= 0 and f2 <= 0 and f3 >= 0 and f4 <=0:
        return True

def is_in_circle(x,y,h,k,r):
    f5 = (x-h)**2 + (y-k)**2 - r**2
    if f5 <= 0:
        return True

def is_in_ellipse(x,y,a,b,h,k):
    f6 = ((x-h)**2)/a**2 + ((y-k)**2)/b**2 - 1
    if f6 <= 0:
        return True

def get_line_equation(x,y,p1,p2):
    if p2[0] - p1[0] != 0:
        m = (p2[1] - p1[1]) / (p2[0] - p1[0])
        f = y - p1[1] - m * x + m * p1[0]
    else:
        f = x - p2[0]
    return f

def is_poly(x,y,scale):
    in_poly1 = False
    in_poly2 = False
    in_poly3 = False
    p1 = [125*scale,94*scale]
    p2 = [163*scale,98*scale]
    p3 = [150*scale,135*scale]
    # Equation of half planes for poly one
    f7 = get_line_equation(x,y,p1,p2)
    f8 = get_line_equation(x,y,p2,p3)
    f9 = get_line_equation(x,y,p1,p3)            
    if f7 >= 0 and f8 <= 0 and f9 <=0:
        in_poly1 = True
    
    p1 = [150*scale,135*scale]
    p2 = [163*scale,98*scale]
    p3 = [163*scale,135*scale]
    # Equation of half planes for poly two
    f10 = get_line_equation(x,y,p1,p2)
    f11 = get_line_equation(x,y,p2,p3)
    f12 = get_line_equation(x,y,p1,p3)            
    if f10 >= 0 and f11 <= 0 and f12 <=0:
        in_poly2 = True
    
    p1 = [163*scale,135*scale]
    p2 = [163*scale,98*scale]
    p3 = [170*scale,60*scale]
    p4 = [193*scale,98*scale]
    p5 = [173*scale,135*scale]
    # Equation of half planes for poly three
    f13 = get_line_equation(x,y,p1,p2)
    f14 = get_line_equation(x,y,p2,p3)
    f15 = get_line_equation(x,y,p3,p4)
    f16 = get_line_equation(x,y,p4,p5)
    f17 = get_line_equation(x,y,p5,p1)
    if f13>=0 and f14>=0 and f15>=0 and f16<=0 and f17<=0:
        in_poly3 = True
        
    return in_poly1 or in_poly2 or in_poly3
    
# Draw obstacles using obstacle points
def draw_obstacles(obstacle_set):
    for p in obstacle_set:
        pxarray[p[0], p[1]] = pygame.Color(0, 0, 0)
        pygame.display.update()

        
def draw_obstacles_bg(img,obstacle_set):
    for p in obstacle_set:
        img[p[1],p[0]] = 0
        
### Apply minkowski sum to obstacle points 
def minkowski_sum(bg,r,obstacle_set,width,height):
    new_obstacle_set = set()
    for p in obstacle_set:
        h = p[0]
        k = p[1]
        # skip convolution for pixels that are completely inside the circle
        if bg[k,h-r] == 0 and bg[k,h+r] == 0 and bg[k-r,h] == 0 and bg[k+r,h] == 0:
            continue
        for x in range(h-r,h+r):
            for y in range(k-r,k+r):
                if x < 0 or x >= width or y < 0 or y >= height:
                    continue
                else:
                    f = (x-h)**2 + (y-k)**2 - r**2
                    if f <= 0:
                        new_obstacle_set.add((x,y))
                    
    return new_obstacle_set
    # using each point find new obstacle points
    # don't forget to delete the old one

In [3]:
import time
# scale the window size
scale = 1
height = 150*scale
width = 250*scale

# create a background image
bg = np.zeros((height,width),dtype=np.uint8)
for x in range(0,bg.shape[0]):
    for y in range(0,bg.shape[1]):
        bg[x,y] = 255

# Define obstacle point x and y set
obstacle_set = set()


#start_time = time.time()
# iterate for each pixel and find out if it on obstacle
# if it is in the obstacle store it in the obstacle set
for x in range(0,width):
    for y in range(0,height):
        if is_in_rect(x,y,[50*scale,38*scale],[100*scale,83*scale]) or is_in_circle(x,y,190*scale,20*scale,15*scale) or is_in_ellipse(x,y,30*scale,12*scale,140*scale,30*scale) or is_poly(x,y,scale):
            obstacle_set.add((x,y))
            bg[y,x] = 0  
#elapsed_time = time.time() - start_time
#print(elapsed_time)
# Calculate Mikowski space
r = 10
obstacle_set = minkowski_sum(bg,r*scale,obstacle_set,width,height)
draw_obstacles_bg(bg,obstacle_set)
#elapsed_time = time.time() - start_time
#print(elapsed_time)
#cv2.imshow("image",bg)
cv2.imwrite("background.png",bg)


# initialize the pygame
ret = pygame.init()
surface = pygame.display.set_mode((width,height))
pygame.display.set_caption('Search Algorithms')

# Define color tuples
white = (255,255,255)
black = (0,0,0)
red = (255,0,0)
green = (0,255,0)
#surface.fill(white)

# set the background image
pygame_bg = pygame.image.load('background.png')
surface.blit(pygame_bg,(0,0))

clock = pygame.time.Clock()
pygame.display.update()
gameExit = False
x = 0
y = 0
while not gameExit:
    for event in pygame.event.get():
        # if close button is clicked close
        if event.type == pygame.QUIT:
            gameExit = True
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                x -= 10
            if event.key == pygame.K_RIGHT:
                x += 10
            if event.key == pygame.K_UP:
                y -= 10
            if event.key == pygame.K_DOWN:
                y += 10
        surface.blit(pygame_bg,(0,0))        
        pygame.draw.rect(surface,green,[x,y,50,50])
        pygame.display.update()
    clock.tick(10)
pygame.quit()

In [60]:
bg.dtype

dtype('uint8')